# Run MLFlow Model in Seldon Core

This notebook shows how you can easily train a model using [MLFlow](https://mlflow.org/) and serve requests within Seldon Core on Kubernetes.

Dependencies

  * ```pip install seldon-core```
  * ```pip install mlflow```
  


## Train Example MlFlow Model

In [ ]:
!git clone https://github.com/mlflow/mlflow

In [ ]:
!python mlflow/examples/sklearn_elasticnet_wine/train.py

## Test Inference Locally

In [ ]:
!pygmentize MyMlflowModel.py

In [ ]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.5-SNAPSHOT mlflow_model:0.1

In [ ]:
!docker run --name "mlflow_model" -d --rm -p 5000:5000 mlflow_model:0.1

In [ ]:
!curl -H "Content-Type: application/x-www-form-urlencoded" -g 0.0.0.0:5000/predict -d 'json={"data":{"names":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"ndarray":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}}'

In [ ]:
!curl -H "Content-Type: application/x-www-form-urlencoded" -g 0.0.0.0:5000/predict -d 'json={"data":{"ndarray":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}}'

In [ ]:
!docker rm mlflow_model --force

## Test Inference on Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory 4096

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.5-SNAPSHOT mlflow_model:0.1

In [ ]:
!kubectl create -f deployment.json

In [ ]:
!kubectl get seldondeployments mlflow-example -o jsonpath='{.status}' 

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete